# init AD9174 with external sampling clock
Direct drive of the DAC. No additonal VCO. Best configuration for phase noise.

Needs an external 5.12 GHz DAC sampling clock connected to J34 (`CLKIN`) of the `AD9174-FMC_EBZ` board. Make sure C34 and C35 are populated. Connect J3 and J41 with a short SMA cable. See `../doc/dac_clocking.png` for details.

requires litex_server running and connected to the VC707 USB-uart port:

```bash
litex_server --uart --uart-port /dev/ttyUSB0 --uart-baudrate 115200

```

In [106]:
import sys
from numpy import *
from time import sleep
from litex import RemoteClient
from ad9174 import Ad9174Settings, Ad9174Init
from hmc7044 import Hmc7044

def getId(r):
    s = ""
    for i in range(64):
        temp = r.read(r.bases.identifier_mem + i * 4)
        if temp == 0:
            break
        s += chr(temp & 0xFF)
    return s

def big_read(r, addr, length, chunk_size=255):
    """
    read data of arbitrary length in chunks
    r = litex RemoteClient object
    addr = start address in [bytes], should be 32 bit aligned
    length = number of 32 bit words to read
    chunk_size = how many words to read in one Etherbone transaction
    """
    dats = []
    while length > 0:
        temp = r.read(addr, min(chunk_size, length))
        dats.append(temp)
        addr += len(temp) * 4
        length -= len(temp)
    # return hstack(dats)
    return [i for dat in dats for i in dat]

def chunks(l, n):
    n = max(1, n)
    return (l[i:i+n] for i in range(0, len(l), n))

def big_write(r, addr, dats, chunk_size=255):
    """
    write data of arbitrary length in chunks
    r = litex RemoteClient object
    addr = start address in [bytes], should be 32 bit aligned
    dats = list of 32 bit integers
    chunk_size = how many words to read in one Etherbone transaction
    """
    if type(dats) == ndarray:
        dats = dats.tolist()
    for c in chunks(dats, chunk_size):
        r.write(addr, c)
        addr += len(c) * 4
        
def hd(dat, pad_width=1, word_width=None):
    ''' print a hex-dump, word_width in bytes '''
    if word_width is None:
        word_width = pad_width
    for i, d in enumerate(dat):
        if i % 8 == 0 and len(dat) > 8:
            print('\n{:04x}: '.format(i * word_width), end='')
        print('{:0{ww}x} '.format(d, ww=pad_width * 2), end='')
    print()

In [174]:
settings = Ad9174Settings(json_file='../build/csr.json')
print(settings)

----------------
 JESD mode 20
----------------
INTERP_CH: 1  INTERP_MAIN: 1  DSP_CLK_DIV: 16
JESD204BSettings(): 5a 05 00 87 00 1f 00 0f 2f 23 80 00 00 e6 
         DID:  90        BID:   5     ADJCNT:   0        LID:   0 
       PHADJ:   0     ADJDIR:   0          L:   8        SCR:   1 
           F:   1          K:  32          M:   1          N:  16 
          CS:   0         NP:  16  SUBCLASSV:   1          S:   4 
       JESDV:   1         CF:   0         HD:   1       RES1:   0 
        RES2:   0       FCHK: 230 
   [ LINK_DW:  32     FR_CLK:   4 ]


In [210]:
r = RemoteClient(csr_csv='../build/csr.csv', debug=False, port=1234)
r.open()
getId(r)

'AD9174 + VC707 test 2021-03-04 17:09:33'

In [197]:
ad = Ad9174Init(r, settings)
wr = ad.ad.wr
rr = ad.ad.rr

# 1. init AD9174

In [116]:
ad.init_ad9174(ADC_CLK_DIV=4, USE_PLL=False)

AD917X_NVM_BLR_DONE: 1
PROD_ID: 0x9174
PROD_GRADE: 0  DEV_REVISION: 5
DLL locked: 1
SPI_PAGEINDX: 0b01000001
CAL_STAT: 1
SERDES PLL locked: 1
MODE_NOT_IN_TABLE: 0


# 2. init HMC7044

In [117]:
hmc = ad.hmc
hmc.init_hmc7044_ext()

clk_div = ad.settings.DSP_CLK_DIV // 4
hmc.setup_channel(12, clk_div, sync_en=False)     # DEV_CLK to the FPGA

# for litejesd, SYSREF must be an integer multiple of the LMFC
lmfc_cycles = ad.settings.K // ad.settings.FR_CLK
hmc.setup_channel(3, clk_div * lmfc_cycles * 10)   # SYSREF (DAC)
hmc.setup_channel(13, clk_div * lmfc_cycles * 10)  # SYSREF (FPGA)
# hmc.trigger_reseed()
hmc.trigger_div_reset()

In [118]:
ad.fpga_print_clocks()

f_jesd = 312.501177 MHz  f_ref = 3.906265 MHz


# 3. Init the FPGA side

In [119]:
# r.regs.ctrl_reset.write(1)  # resets ALL clockdomains (HARSH!)
r.regs.control_control.write(0b01)  # resets PHYs and jesd core
print('status: {:03b}'.format(r.regs.control_status.read()))

# bit1: links_enable,  bit0: phys_reset
r.regs.control_control.write(0b10)
# bit2: /jsync,  bit1: links_ready,  bit0: phys_ready
print('status: {:03b}'.format(r.regs.control_status.read()))

ad.trigger_jref_sync()
ad.print_irq_flags(True, True)
if ad.print_irq_flags(True):
    print('😭')
else:
    print('😃')

status: 000
status: 111
SYNC_ROTATION_DONE 1
DYN_LINK_LATENCY  2 cycles
😃


# 4. Test link

In [13]:
ad.print_irq_flags(True)
print()

ad.print_ilas()

ad.print_lane_status()
print()

ad.test_stpl()


JESD settings, received on lane 0 vs (programmed):
450: 5a (5a)
451: 05 (05)
452: 00 (00)
453: 87 (87)
454: 00 (00)
455: 1f (1f)
456: 00 (00)
457: 0f (0f)
458: 2f (2f)
459: 23 (23)
45a: 80 (80)
45b: 00 (00)
45c: 00 (00)
45d: e6 (e6)
CHK: e6 (e6) 

Lane status:
      LANE_DESKEW: 11111111
    BAD_DISPARITY: 00000000
     NOT_IN_TABLE: 00000000
 UNEXPECTED_KCHAR: 00000000
    CODE_GRP_SYNC: 11111111
       FRAME_SYNC: 11111111
    GOOD_CHECKSUM: 11111111
   INIT_LANE_SYNC: 11111111
FIFO_STATUS_REG_0: 00000000
FIFO_STATUS_REG_1: 00000000
fpga j_sync errs: 94202095

STPL test:
converter: 0, sample: 0, tp: 597a, fail: 0
converter: 0, sample: 1, tp: b2f3, fail: 0
converter: 0, sample: 2, tp: 0c6c, fail: 0
converter: 0, sample: 3, tp: 65e5, fail: 0
converter: 0, sample: 4, tp: bf5e, fail: 0
converter: 0, sample: 5, tp: 18d7, fail: 0
converter: 0, sample: 6, tp: 7250, fail: 0
converter: 0, sample: 7, tp: cbc9, fail: 0
converter: 0, sample: 8, tp: 2542, fail: 0
converter: 0, sample: 9, tp: 7eb

0

# IQ sample waveforms
kinda works. Output can be observed on IQ analyzer / scope.

In [198]:
wr(0x596, (1 << 3) | (1 << 2))  # Turn ON Transmit enable
r.regs.sample_gen_max_ind.write(32)

In [208]:
# Zero out the sample memories
# TODO: this seems to crash litex_server :(
for s in range(16):
    mem = getattr(r.mems, f'm0_s{s}')
    depth = mem.size // 4
    big_write(r, mem.base, [0] * depth)

In [225]:
# Read a memory
dat = big_read(r, r.mems.m0_s0.base, r.mems.m0_s0.size // 4)
hd(dat, 2)


0000: 0000 0080 0100 0180 0200 0280 0300 0380 
0010: 0400 0480 0500 0580 0600 0680 0700 0780 
0020: 0800 0880 0900 0980 0a00 0a80 0b00 0b80 
0030: 0c00 0c80 0d00 0d80 0e00 0e80 0f00 0f80 
0040: 1000 1080 1100 1180 1200 1280 1300 1380 
0050: 1400 1480 1500 1580 1600 1680 1700 1780 
0060: 1800 1880 1900 1980 1a00 1a80 1b00 1b80 
0070: 1c00 1c80 1d00 1d80 1e00 1e80 1f00 1f80 
0080: 2000 2080 2100 2180 2200 2280 2300 2380 
0090: 2400 2480 2500 2580 2600 2680 2700 2780 
00a0: 2800 2880 2900 2980 2a00 2a80 2b00 2b80 
00b0: 2c00 2c80 2d00 2d80 2e00 2e80 2f00 2f80 
00c0: 3000 3080 3100 3180 3200 3280 3300 3380 
00d0: 3400 3480 3500 3580 3600 3680 3700 3780 
00e0: 3800 3880 3900 3980 3a00 3a80 3b00 3b80 
00f0: 3c00 3c80 3d00 3d80 3e00 3e80 3f00 3f80 
0100: 4000 4080 4100 4180 4200 4280 4300 4380 
0110: 4400 4480 4500 4580 4600 4680 4700 4780 
0120: 4800 4880 4900 4980 4a00 4a80 4b00 4b80 
0130: 4c00 4c80 4d00 4d80 4e00 4e80 4f00 4f80 
0140: 5000 5080 5100 5180 5200 5280 5300 5380 
0150: 5400 5

1890: 0000 0000 0000 0000 0000 0000 0000 0000 
18a0: 0000 0000 0000 0000 0000 0000 0000 0000 
18b0: 0000 0000 0000 0000 0000 0000 0000 0000 
18c0: 0000 0000 0000 0000 0000 0000 0000 0000 
18d0: 0000 0000 0000 0000 0000 0000 0000 0000 
18e0: 0000 0000 0000 0000 0000 0000 0000 0000 
18f0: 0000 0000 0000 0000 0000 0000 0000 0000 
1900: 0000 0000 0000 0000 0000 0000 0000 0000 
1910: 0000 0000 0000 0000 0000 0000 0000 0000 
1920: 0000 0000 0000 0000 0000 0000 0000 0000 
1930: 0000 0000 0000 0000 0000 0000 0000 0000 
1940: 0000 0000 0000 0000 0000 0000 0000 0000 
1950: 0000 0000 0000 0000 0000 0000 0000 0000 
1960: 0000 0000 0000 0000 0000 0000 0000 0000 
1970: 0000 0000 0000 0000 0000 0000 0000 0000 
1980: 0000 0000 0000 0000 0000 0000 0000 0000 
1990: 0000 0000 0000 0000 0000 0000 0000 0000 
19a0: 0000 0000 0000 0000 0000 0000 0000 0000 
19b0: 0000 0000 0000 0000 0000 0000 0000 0000 
19c0: 0000 0000 0000 0000 0000 0000 0000 0000 
19d0: 0000 0000 0000 0000 0000 0000 0000 0000 
19e0: 0000 00

In [214]:
# Generate a triangle wave pattern
dat = arange(4096) << 3
dat

array([    0,     8,    16, ..., 32744, 32752, 32760])

In [226]:
# Interlace the smaples from `dat` into all 16 memories
for s in range(16):
    mem = getattr(r.mems, f'm0_s{s}')
    depth = mem.size // 4
    big_write(r, mem.base, dat[s::16])
    
r.regs.sample_gen_max_ind.write(len(dat) // 16)

# Something's not right with the memory

In [192]:
# a little broken maybe?
def write_samples(vals, m=0):
    ''' vals = [-1.0 ... 1.0], m = converter index '''
    bits = 16
    vals = (array(vals) * 0x7fff).astype(int)
    isNeg = vals < 0
    vals[isNeg] = (vals[isNeg] + (1 << bits)) & ((1 << bits) - 1)
    vals = vals.tolist()
    
    for s in range(16):
        mem = getattr(r.mems, f'm0_s{s}')
        depth = mem.size // 4
        big_write(r, mem.base, vals[s::16])
    
    r.regs.sample_gen_max_ind.write(len(vals) // 16 - 1)

In [195]:
write_samples(zeros(255), 0)
# sleep(0.5)
# write_samples(zeros(255), 1)

In [193]:
write_samples(linspace(-0.9, 0.9, 254), 0)

In [194]:
write_samples(sin(linspace(0, 3 * pi, 254)), 1)

In [50]:
write_samples(sin(linspace(0, 2.5 * pi, 254)), 0)

# Main DDS

In [35]:
# Setup DDSes
wr(0x1E6, (1 << 1))             # Enable DDSM_EN_CAL_DC_INPUT (see Fig. 80) (tone on / off)
wr(0x112, (1 << 3) | (1 << 2))  # Enable NCO + Modulus
wr(0x596, (1 << 3) | (1 << 2))  # Turn ON Transmit enable

# setup main DDS frequency and amplitude
ad.setTone(1, 1e9, 1, f_ref=5.12e9)

DC_CAL_TONE: ff 50 
DDSM_FTW: 00 00 00 00 00 32 
DDSM_FTW_LOAD_ACK: 1


In [36]:
# DC test-mode off, enable JESD input
ad.ad.wr(0x1e6, 0)

0

In [53]:
# Adjust DDS phase offset
ad.setTone(1, phase=0)

DDSM_NCO_PHASE_OFFSET: 00 00 


# app layer PRBS 😭
doesn't work 😠 this feature seems completely undocumented for AD9174 ☹️

In [60]:
r.regs.prbs_gen_sample_prbs_en.write(0)

In [12]:
r.regs.prbs_gen_sample_prbs_en.read()

0

In [79]:
ad.ad.help(0x14b)

reg 0x14b, PRBS:
    bit 7, PRBS_GOOD_Q, R, reset 0x00
    DAC1 good data indicator. 1: Correct PRBS sequence detected. 0: Incorrect sequence detected. Sticky; reset to 1 by PRBS_RESET.

    bit 6, PRBS_GOOD_I, R, reset 0x00
    DAC0 good data indicator. 0: Incorrect sequence detected. Sticky; reset to 1 by PRBS_RESET. 1: Correct PRBS sequence detected.

    bit 5, RESERVED, R, reset 0x00
    Reserved.

    bit 4, PRBS_INV_Q, R/W, reset 0x01
    DAC1 data inversion. 0: Expect normal data. 1: Expect inverted data.

    bit 3, PRBS_INV_I, R/W, reset 0x00
    DAC0 data inversion. 0: Expect normal data. 1: Expect inverted data.

    bit 2, PRBS_MODE, R/W, reset 0x00
    Select which PRBS polynomial is used for the datapath PRBS test. 0: 7-bit: x7 + x6 + 1. 1: 15-bit: x15 + x14 + 1.

    bit 1, PRBS_RESET, R/W, reset 0x00
    Reset error counters. 0: Normal operation. 1: Reset counters.

    bit 0, PRBS_EN, R/W, reset 0x00
    Enable PRBS checker. 0: Disable. 1: Enable.



In [10]:
ad.ad.wr(0x14b, 0b0111)
ad.ad.wr(0x14b, 0b0101)

for ch in range(7):
    ad.ad.wr(0x14e, ch)
    print('{:d}: {:08b}, {:3d}, {:3d}'.format(ch, ad.ad.rr(0x14b), ad.ad.rr(0x14c), ad.ad.rr(0x14d)))

0: 00000101, 255, 255
1: 00000101, 255, 255
2: 00000101, 255, 255
3: 11000101,   0,   0
4: 11000101,   0,   0
5: 11000101,   0,   0
6: 00000101, 255, 255
